## Импорты и загрузка данных

In [1]:
import numpy as np
import pandas as pd

import os

In [2]:
path = os.getcwd()

fish_df = pd.read_csv(path + r"\data\db1\ref\fish.csv", error_bad_lines=False, sep=';')
prod_designate_df = pd.read_csv(path + r"\data\db1\ref\prod_designate.csv", error_bad_lines=False, sep=';')
prod_type_df = pd.read_csv(path + r"\data\db1\ref\prod_type.csv", error_bad_lines=False, sep=';')
regime_df = pd.read_csv(path + r"\data\db1\ref\regime.csv", error_bad_lines=False, sep=';')
region_df = pd.read_csv(path + r"\data\db1\ref\region.csv", error_bad_lines=False, sep=';')

catch_df = pd.read_csv(path + r'\data\db1\catch.csv', error_bad_lines=False, sep=',')

# Подмена данных + нормализация
def spoofing(id):
    return fish_df[fish_df['id_fish'] == id]['fish'].values[0]
catch_df['fish'] = catch_df['id_fish'].apply(spoofing)
catch_df['catch_volume'] = catch_df['catch_volume']*1000

product_df = pd.read_csv(path + r'\data\db1\product.csv', error_bad_lines=False, sep=',')

ext1 = pd.read_csv(path + r"\data\db2\Ext.csv", error_bad_lines=False, sep=',')
ext2 = pd.read_csv(path + r"\data\db2\Ext2.csv", error_bad_lines=False, sep=',')

In [12]:
fabrico = ext1[ext1['id_fishery'] == -1]
fabrico_vsd = fabrico['id_vsd'].unique().tolist()
ext1

,id_fishery,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
0,2208303,6493928,2022-05-24 00:00:00,\N,5022636,8745666,\N,\N,\N
1,5538439,3846064,2022-05-24 00:00:00,\N,839830,3846313,\N,\N,\N
2,8688535,3107158,2022-05-24 00:00:00,\N,9310751,5215676,\N,\N,\N
3,8688535,3107158,2022-05-24 00:00:00,\N,9310751,4008716,\N,\N,\N
4,9102536,6493928,2022-05-24 00:00:00,\N,2900344,8522784,\N,\N,\N
...,...,...,...,...,...,...,...,...,...
3260793,2140098,7372847,2020-12-30 00:00:00,\N,4176694,2113660,\N,\N,\N
3260794,3458281,7115834,2020-12-29 00:00:00,\N,906811,6111511,\N,\N,\N
3260795,3458281,7115834,2020-12-29 00:00:00,\N,906811,3138810,\N,\N,\N
3260796,3458281,7115834,2020-12-29 00:00:00,\N,906811,2081767,\N,\N,\N


In [ ]:
ext2

In [ ]:
# Чистка больших данных от анонимных данных
ext2 = 

ext1 = ext1.drop(ext1[ext1['id_fishery'] == -1].index)
ext1 = ext1.drop(columns=['Name_Plat', 'Product_period', 'Region_Plat', 'numPart'])



In [178]:
# Сводная таблица по всем поступлениям согласно IdFish и выбранного промежутка времени.
def calculatingSummOfFish(fish_name, date_start, date_end):
    print('Processing incoming values')
    # смотрим по всем выловам этой рыбы
    data = catch_df[catch_df['fish'] == fish_name]
    # уникальные номера кораблей ловящих только эту рыбу
    ships = list(set(data['id_ves'].to_list()))
    
    # создаю датафрейм с владельцами и выловом по судам    
    total_df = pd.DataFrame()
    
    total_ships = []
    total_owners = []
    total_catches = []
    
    # добавляем в общую кашу дату и документируем владельцев
    for ship in ships:
        ship_data = data[data['id_ves'] == ship]
        owners = list(set(ship_data['id_own'].to_list()))
        
        for owner in owners:
            ship_owner_data = ship_data[ship_data['id_own'] == owner]
            
            try:
                ship_owner_end_data = ship_owner_data[ship_owner_data['date'] < date_end]
                ship_owner_start_end_data = ship_owner_end_data[ship_owner_end_data['date'] > date_start]
                total_catch = ship_owner_start_end_data['catch_volume'].sum()
            except e:
                print(e)
            
            if total_catch > 0:
                total_catches.append(total_catch)
                total_ships.append(ship)
                total_owners.append(owner)
            
    total_df['total_catches'] = total_catches
    total_df['ship'] = total_ships
    total_df['owner'] = total_owners
    
    return total_df

In [227]:
# Вспомогательная функция вычисления для КОНКРЕТНЫХ ВЛАДЕЛЬЦЕВ ГРУПП СУДОВ, по FishID
def calculatingSummOfFishTransfering(fish_name, start, end):
    print('Processing transfering')
    total_owners = []
    total_transfers = []
    total_fishery_date = []
    total_vsd_date = []
    
    # Смотрю по каждому ID акты передачи    
    vsd_fish_id = ext2[ext2['fish'] == fish_name]['id_vsd']
    # Для каждого акта определяю владельца     
    for vsd in vsd_fish_id:
        owners = ext1[ext1['id_vsd'] == vsd]['id_own'].unique().tolist()
        # Проверяю является ли владелец связанным по двум базам  
        for owner in owners:
            if owner in market_owners:
                data = ext2[ext2['id_vsd'] == vsd]
                try:
                    total_owners.append(owner)
                    total_transfers.append(data['volume'].values[0])
                    total_fishery_date.append(data['date_vsd'].values[0][5:10])
                    total_vsd_date.append(ext1[ext1['id_vsd'] == vsd]['date_fishery'].values[0][5:10])
                except Exception() as e:
                    print(e)
                    
    df = pd.DataFrame()
    
    df['owner'] = total_owners
    df['volume'] = total_transfers
    df['fishery_date'] = total_fishery_date
    df['vsd_date'] = total_vsd_date
    
    return df

### Поиск аномалий

#### Осуществим с помощью вычленения ID рыбы и даты вылова (для простоты возьмем весь период, но можно брать любой промежуток)

In [228]:
fish_id = input('Введите ID рыбы\n')

default_data = (catch_df['date'][0], catch_df['date'][len(catch_df) - 1])
start, end = default_data

Введите ID рыбы
1


#### Возьмем уникальный лист владельцев судов

In [229]:
catcher_owners = catch_df['id_own'].unique()
traider_owners = ext1['id_own'].unique()

market_owners = []
for owner in traider_owners:
    if owner in catcher_owners:
        market_owners.append(owner)  

In [230]:
print(len(catcher_owners), len(traider_owners), len(market_owners))

350 158 121


#### Как видим, db2 не полностью покрывает всех владельцев, следовательно, будем использовать для поиска только тех, которые задействованы в обоих БД

In [231]:
# Смотрим по общему поступлению владельца и общей передачи
def getAllTransfers(id, start, end):
    incoming = calculatingSummOfFish(id, start, end)
    outcoming = calculatingSummOfFishTransfering(id, start, end)
    
    # Важно учитывать неналичие в таблице вылова владельцев которые не передают ничего.
    inputs = incoming['owner'].unique().tolist()
    outputs = outcoming['owner'].unique().tolist()
    
    owners = []
    
    for item in inputs:
        if item in outputs:
            owners.append(item)
    
    print(owners)
    
    return incoming, outcoming
    

In [232]:
in_, out = getAllTransfers('минтай', start, end)

Processing incoming values
Processing transfering
[4840022, 3846064, 4769013, 7115834, 7636185, 999889, 2369769, 1546362, 4882414, 7478978, 8769015, 7759328, 8679785, 9290322, 9382283, 6022174, 8870240, 7180396, 6462295, 439595, 6545965, 3101919, 4787153, 7484551, 3415543, 6377225, 3107158, 2680100, 9449188, 1754744, 6948246, 4673799, 3095952, 7629399, 4471486, 2476787]


In [201]:
transfered = out[out['owner'] == 3846064]['volume'].sum()

In [206]:
catched = in_[in_['owner'] == 3846064]['total_catches'].sum()

In [215]:
inputs = in_['owner'].unique().tolist()
outputs = out['owner'].unique().tolist()
owners = []
    
if item in inputs:
    if item in outputs:
        owners.append(item)

TypeError: list indices must be integers or slices, not str

In [216]:
len(out['owner'].uniqueque())

TypeError: list indices must be integers or slices, not str

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit
0,7230988,8934601,400,минтай,2022-01-01 00:39:56,75480,\N
1,3420538,1439843,400,минтай,2022-01-01 01:34:08,62586,\N
2,9509257,342637,292,треска,2022-01-01 01:34:11,1155,\N
3,1894307,1801854,400,минтай,2022-01-01 01:34:05,111048,\N
4,9410489,3126516,88,окунь-клювач,2022-01-01 01:34:11,1844,\N
...,...,...,...,...,...,...,...
3115275,7436163,4038632,294,краб-стригун опилио,2022-05-25 12:54:52,774,\N
3115276,4519348,9520759,294,краб-стригун опилио,2022-05-25 12:57:18,5533,\N
3115277,1445226,7953654,292,треска,2022-05-25 12:19:02,8235,\N
3115278,7624369,1390453,408,пикша,2022-05-25 12:19:02,756,\N
